# Retinotopic Maps and Model-based Analysis of fMRI

## Introduction

**Author**: Noah C. Benson &lt;[nben@nyu.edu](mailto:nben@nyu.edu)&gt;  
**Repository**: [https://github.com/noahbenson/neurohackademy2019](https://github.com/noahbenson/neurohackademy2019)

This notebook is an introduction to retinotopic maps and model-based analysis of fMRI, created for the Neurohackademy 2019 summer course by Noah C. Benson. In this notebook, we will use Python and in particular the [neuropythy](https://github.com/noahbenson/neuropythy) and [popeye](https://github.com/kdesimone/popeye) open source libraries to download both anatomical/structural data and functional data (BOLD time-series) from the [Human Commectome Project](https://db.humanconnectome.org/) then to analyze the time-series. This notebook is intended to be a verbose description of the various tools and techniques involved in such an analysis; it is likely that we will not cover all the material in the notebook during the Neurohackademy lecture.

The contents of this notebook are as follows:

* **Introduction**  
  This section gives a brief introduction to the project in the notebook.
* **Initialization**  
  In this section of the notebook, we import libraries, configure the plotting environment, and declare custom functions that we will need throughout the notebook. We will also verify that you have configured your environment correctly for this notebook to work.
* **Demo 1**: Plot Structural data from the HCP.
  Here, we will load the relevant anatomical data from the HCP's Amazon S3 interface using [neuropythy](https://github.com/noahbenson/neuropythy) then examine and verify those data by plotting them.
* **Demo 2**: Solve a pRF model and plot the results.  
  This section documents population receptive field (pRF) models and demonstrates how to fit a pRF model from the fMRI data obtained in the previous section. We will use pRF data from the HCP that has been preprocessed and prepared for this tutorial.
* **Demo 3**: Plot retinotopic maps.
  Finally, we examine retinotopic maps--the organization of pRFs across the cortical surface.

## Initialization

Here, we import a number of libraries and configure matplotlib/pyplot, our 2D plotting tool. We then check that neuropythy is able to communicate with the Amazon S3.

### Import and Configure Libraries

In [ ]:
# Import some standard/utility libraries:
import os, sys, time, h5py, zipfile
import six           # six provides python 2/3 compatibility

# Import our numerical/scientific libraries, scipy and numpy:
import numpy as np
import scipy as sp

# The pimms (Python Immutables) library is a utility library that enables lazy
# computation and immutble data structures; https://github.com/noahbenson/pimms
import pimms

# The popeye library (https://github.com/kdesimone/popeye) is a pRF modeling
# library that we will use to obtain pRF parameters for the functional data
import popeye

# The neuropythy library is a swiss-army-knife for handling MRI data, especially
# anatomical/structural data such as that produced by FreeSurfer or the HCP.
# https://github.com/noahbenson/neuropythy
import neuropythy as ny

# Import graphics libraries:
# Matplotlib/Pyplot is our 2D graphing library:
import matplotlib as mpl
import matplotlib.pyplot as plt
# We also use the 3D graphics library ipyvolume for 3D surface rendering
import ipyvolume as ipv

In [ ]:
# These "magic commands" tell matplotlib that we want to plot figures inline and
# That we are using qt as a backend; due to bugs in certain versions of
# matplotlib, we put them in a separate cell as the import statements above and
# the configuration statements below.
%gui qt
%matplotlib inline

In [ ]:
# Additional matplotlib preferences:
font_data = {'family':'sans-serif',
             'sans-serif':['Helvetica Neue', 'Helvetica', 'Arial'],
             'size': 10,
             'weight': 'light'}
mpl.rc('font',**font_data)
# we want relatively high-res images, especially when saving to disk.
mpl.rcParams['figure.dpi'] = 72*2
mpl.rcParams['savefig.dpi'] = 72*4

### Check neuropythy's Amazon S3 Configuration

The [neuropythy](https://github.com/noahbenson/neuropythy) library can easily be configured to automatically download HCP data it is requested. In order to do this, however, it must have been given a set of HCP credentials. The HCP uses the Amazon S3 so these credentials are in the form of a "key" and a "secret". To obtain HCP credentials, you must register at the [HCP database website](https://db.humanconnectome.org/) then generate Amazon S3 credentials through their interface. The [neuropythy configuration documentation](https://github.com/noahbenson/neuropythy/wiki/Configuration) explains how to do this in more detail.

Your credentials will look something like "`AKAIG8RT71SWARPYUFUS`" and "`TJ/9SJF+AF3J619FA+FAE83+AF3318SXN/K31JB19J4`" (key and secret). They are often printed with a "`:`" between them like "`AKAIG8RT71SWARPYUFUS:TJ/9SJF+AF3J619FA+FAE83+AF3318SXN/K31JB19J4`". If, when you started up the docker-container running this notebook, you provided your credentials as a `:`-separated string like this via the environment variable HCP_CREDENTIALS, you should be fine. If neuropythy found your credentials, they should be in neuropythy's `config` structure, which behaves like a Python dictionary. However, if you are running this tutorial in the Neurohackademy 2019 class itself, you likely were not able to do this and will have to set these credentials manually. If neuropythy could not read your credentials or if they were not set, then `ny.config['hcp_credentials']` will be `None`. If this is the case, you can either:
 * set the credentials directly in this notebook by running something like:  
   ```python
   key = 'AKAIG8RT71SWARPYUFUS'
   secret = 'TJ/9SJF+AF3J619FA+FAE83+AF3318SXN/K31JB19J4'
   ny.config['hcp_credentials'] = (key, secret)
   ```
 * restart the docker container after configuring the `HCP_CREDENTIALS` environment variable:
   ```bash
   > export HCP_CREDENTIALS="AKAIG8RT71SWARPYUFUS:TJ/9SJF+AF3J619FA+FAE83+AF3318SXN/K31JB19J4"
   > docker-compose up
   ```
 * store the credentials in a local file and import its contents into the `HCP_CREDENTIALS` environment variable:
   ```bash
   > echo "AKAIG8RT71SWARPYUFUS:TJ/9SJF+AF3J619FA+FAE83+AF3318SXN/K31JB19J4" > ~/.hcp-passwd
   > export HCP_CREDENTIALS="`cat ~/.hcp-passwd`"
   > docker-compose up
   ```

We also want to check that neuropythy was able to connect to the HCP database. If this fails, it is possible that either your credentials are incorrect/expired or that you do not have a valid internet connection, or that you did something unexpected when mounting volumes into the docker that prevent neuropythy from knowing where to store the HCP data (unlikely).

In [ ]:
# Check that HCP credentials were found:
if ny.config['hcp_credentials'] is None:
    raise Exception('No valid HCP credentials were found!\n'
                    'See above instructions for remedy.')

# Check that we can access the HCP database:
# To do this we grab the 's3fs' object from neuropythy's 'hcp' dataset; this
# object maintains a connection to Amazon's S3 using the hcp credentials. We use
# it to perform a basic ls operation on the S3 filesystem. If this fails, we do
# not have a working connection to the S3.
try: files = ny.data['hcp'].s3fs.ls('hcp-openaccess')
except Exception: files = None
if files is None:
    raise Exception('Could not communicate with S3!\n'
                    'This probably indicates that your credentials are wrong'
                    ' or that you do not have an internet connection.')

print('Configuration appears fine!')

## Demo 1: Load a Subject and Plot their Pial Surfaces

In this section of the tutorial we will choose a subject from the HCP and plot structural data from that subject.

In many projects, the task of wrangling data into a format that enables your analysis is 98% of the work, but becomes only 2% of the resulting scientific paper. When dealing with fMRI data, this difficulty often arises from the complex and often poorly-documented formats and directory structures used to store the data. Both FreeSurfer and the HCP Pipelines are capable of processing a T1-weighted anatomical image into a rich set of (mostly equivalent) information about the cortical surface of the brain, its anatomy, and how its geometry maps onto anatomical atlases (like FreeSurfer's *fsaverage* subject or the HCP's *fs_LR* hemispheres). However, interpreting these data is a chore, and, even if you understand the directories and formats, the amount of code required to load the relevant data for an analysis can be quite substantial.

One of the main goals of neuropythy is to reduce the burden of both understanding and coding access to these data. For the human connectome project in particular, neuropythy will automatically download and manage structural data as if it were all always on your local storage without you needing to think about it. This section will demo these abilities.

### Choosing a sample subject

Before we run the following demos, we need to choose an example HCP subject to use. I have chosen one below, but you should be able to use basically any HCP subject you like, so long as they are in the HCP_1200 release. To see a list of these subjects, you can ask neuropythy:

In [ ]:
# Get the first 10 HCP_1200 subjects:
ny.hcp.subject_ids[:10]

In [ ]:
# How many HCP_1200 subjects are there?
len(ny.hcp.subject_ids)

Since we are going to be doing analysis on the retinotopic maps, we will eventually want to choose a subject that was part of the retinotopic mapping experiments. To get a list of these subjects, we can ask neuropythy's 'hcp_retinotopy' dataset for its list:

In [ ]:
# How many retinotopy subjects?
len(ny.data['hcp_retinotopy'].subject_ids)

Note that there are actually only 181 HCP subjects with retinotopy, but the dataset includes three additional subjects: subject 999999 (group average of all 181 subjects) and subjects 999998 and 999997 (group average of split halves of the 181 subjects).

In [ ]:
# Choose the subject we will use for the following demos
sid = 100610

# Make sure that's a real subject
if sid not in ny.hcp.subject_ids:
    raise Exception('Subject %s is not an HCP_1200 subject!' % (sid,))
elif sid not in ny.data['hcp_retinotopy'].subject_ids:
    print('Subject %s is not a retinotopy subject!' % (sid,))

### Plot some structural data

Here we will load the HCP subject we picked above using neuropythy; neuropythy will give us a subject object that encapsulates a large amount of data about the subject, including volumetric MR images as well as surface-based data. We will focus mostly on the surface data.

In [ ]:
# Get the subject object:
sub = ny.hcp_subject(sid)

# We want to plot both the LH and the RH pial surfaces; this code uses the 'lh'
# and 'rh' hemispheres which are the subject's "native" hemispheres. The HCP
# also includes a number of "atlas" hemispheres that are roughly equivalent but
# that have been coerced to a common geometrical tesselation. These hemispheres
# contain less accurate representations of the subject's structural data but
# have the advantage that vertices on the surface are approximately aligned
# between subjects. For a list of all supported hemispheres, including atlas
# hemispheres, you can see sub.hemis, which is a dictionary of all the known
# hemisphere objects. sub.lh and sub.rh are aliases for sub.hemis['lh'] and
# sub.hemis['rh'].

surf = 'pial'       # we're plotting the pial surface
view = 'posterior'  # we want a view of the posterior of the brain

# make the plot; because we are passing in 3D surfaces, it returns an 
# ipyvolume figure (which we will not actually use)
ny.cortex_plot((sub.lh.surfaces[surf], sub.rh.surfaces[surf]), view=view)

# tell ipyvolume to show the plot:
ipv.show()

#### Plot Structural Properties

Geometric structural data, such as the meshes plotted in the section above, are only one kind of data that are included in FreeSurfer and HCP subject directories. Neuropythy is capable of loading most of these data, and automatically organizes them, when possible, into properties that are associated with the hemisphere objects and surface meshes. An example of this is the cortical thickness (in mm).

In [ ]:
# The cortical thickness is actually just a vector of values, one per vertex in
# the hemisphere's mesh representation. Properties are stored in numpy arrays
# and can be accessed easily:
lh_thickness = sub.lh.prop('thickness')
print('Property shape:    ', lh_thickness.shape)
print('Mesh vertex count: ', sub.lh.vertex_count)

# Make a histogram of the values:
(fig,ax) = plt.subplots(1,1, figsize=(6,4), dpi=72*4)
ax.hist(lh_thickness)
ax.set_xlabel('Cortical Thickness [mm]')
ax.set_ylabel('Frequency [vertices]')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

In [ ]:
# We can replot the 3D plot above using the cortical thickness as the overlay
# for the surface; this time we will use the white surface instead of the pial:

surf = 'white'
view = 'posterior'

# Note: the string 'thickness' can alternately be a vector of numbers--i.e.,
# the thickness values for each vertex rather than the name of the property
ny.cortex_plot((sub.lh.surfaces[surf], sub.rh.surfaces[surf]),
               view=view, color='thickness', cmap='hot')

# tell ipyvolume to show the plot:
ipv.show()

## Demo 2: Fit a pRF model to BOLD data

The HCP performed retinotopic mapping experiments on 181 subjects. The data from these maps are available via the HCP's S3 interface, but only in unprocessed volumetric form. Rather than preprocess the volumes, we will use some preprocessed data from the HCP that has already been prepared and included in the repository for this tutorial. Preprocessed 7T functional data-files can be downloaded from [the Connectome Database](https://db.humanconnectome.org/).

### The BOLD Data

The BOLD time-series data we will be using has been preprocessed and prepared for the course; these data can be downloaded from the https://db.humanconnectome.org/ database website as CIFTI files. Because these files can be very large, we will just load part of a time-series from a JSON file that is part of the tutorial GitHub repository.

In [ ]:
bold_data = ny.load('data/example_BOLD.json')
signal = bold_data['timeseries']
(fig,ax) = plt.subplots(1,1, figsize=(7,2.5), dpi=2*72)
ax.plot(np.arange(len(signal)), signal, 'k.-', lw=0.5, ms=1, marker='.')

ax.set_xlim([0,600])
ax.set_xlabel('Time [s]')
ax.set_ylabel('BOLD Signal [AU]')
for k in ['top','right']: ax.spines[k].set_visible(False)
# Adjust the x-axis to be nicer for glancing at time:
ax.set_xticks(np.arange(0,601,300))
ax.xaxis.set_minor_locator(mpl.ticker.MultipleLocator(60))

### The Stimulus Data

In order to interpret the BOLD time-series data from the retinotopy experiment, we will also need to know what the stimulus looked like, approximately. The following cell loads the stimulus and rearranges it to be (rows x cols x frames).

In [ ]:
# The stimulus data comes from a MAT file (really an HDF5 file) that is bundled
# with the github repository for this tutorial; this file can be found on the
# OSF page for the HCP 7T Retinotopy Dataset (https://osf.io/bw9ec/) in the
# apertures.zip file.
with h5py.File('data/RETBARsmall.mat', 'r') as fl:
    stimulus_array = np.asarray(fl['stim'])
# We swap rows/cols here because matlab stores things in column-major order
stimulus_array = np.transpose(stimulus_array / 255.0, [2,1,0])
# Since we are looking at BOLD time-series data from 2 runs of this stimulus, we
# need to duplicate the stimulus array along the time axis (the third axis)
stimulus_array = np.concatenate([stimulus_array, stimulus_array], axis=2)
stimulus_array.shape

In [ ]:
# Total frames in the stimulus
frames = stimulus_array.shape[-1]

# Setup plot for select frames of the stimulus:
(fig,axs) = plt.subplots(6,10, figsize=(14,9), dpi=72*2)
axs = axs.flatten()
n = len(axs) # number of frames we'll plot

frame_nos = np.round(np.linspace(0, frames-1, n)).astype('int')
for (fno,ax) in zip(frame_nos, axs):
    ax.imshow(stimulus_array[:,:,fno], vmin=0, vmax=1, cmap='gray')
    ax.axis('off')
    ax.set_title('Frame %03d' % (fno,))

### Fitting the pRF Model using popeye

In this cell we use popeye's "Ordinary Gaussian" model with a fitted HRF to solve for the pRF parameters. Popeye does this by performing a "ballpark" grid search then taking the best grid-search result and performing a non-linear minimization from this starting point.

In [ ]:
import popeye.og_hrf as og
from popeye.visual_stimulus import VisualStimulus

# Wrap the stimulus array up in Popeye's stimulus interpreter; it needs a bit of
# additional information to, for example, convert from pixels to degrees:
screen_dist         = 100   # distance to screen...
screen_width        = 28.11 # with screen_dist=100, this gives a 16° aperture
ballpark_downsample = 0.5   # downsample stimulus frames by this much for quick estimates
tr_length           = 1.0   # scan time (seconds) between BOLD images in the bold signal

stimulus = VisualStimulus(stimulus_array[:,:,-600:], screen_dist, screen_width,
                          ballpark_downsample, tr_length, 'short')
hrf   = popeye.utilities.double_gamma_hrf
model = og.GaussianModel(stimulus, hrf)
model.hrf_delay = -0.25 # this must be set manually for _hrf models

# popeye uses a sparse grid-fit followed by a fine-detail nonlinear fit; for both of these
# we want to specify boundaries for the searches. These are (min,max) parameter values for
# the parameters x0, y0, sigma, and hrf_delay (in that order).
# For the grid-fit, we use slightly less extreme limits so that we don't start at extreme
# positions in the nonlinear fit:
ballpark_bounds = {'x':(-8, 8),  'y':(-8, 8),  'sigma':(0.20,6),  'hrf':(-2,2)}
estimate_bounds = {'x':(-16,16), 'y':(-16,16), 'sigma':(0.01,12), 'hrf':(-7,7)}

# The fitting process will take a few minutes
param_order = ['x','y','sigma','hrf']
fit = og.GaussianFit(model, signal[-600:],
                     [ballpark_bounds[k] for k in param_order],
                     [estimate_bounds[k] for k in param_order],
                     Ns=4)

In [ ]:
# Print the found pRF parameters:
text = r'''
PRF Parameters:
    (x0, y0): (%4.2f, %4.2f),
       sigma: %4.2f
        gain: %4.2f
   HRF delay: %4.2f
   r-squared: %4.2f
''' % (fit.x, fit.y, fit.sigma, fit.beta, 5+fit.hrf_delay, fit.rsquared)

# We'll now plot a visualization of the pRF itself...
(fig,ax) = plt.subplots(1,1, figsize=(7,7))

# Sample the visual field at points from -10°, to 10°:
max_eccen = 10
res = 250
samples = np.linspace(-max_eccen, max_eccen, res)
(x,y) = np.meshgrid(samples, samples)
# get the gaussian pRF sensitivity values:
(x0,y0,sigma) = (fit.x, fit.y, fit.sigma)
z = np.exp(-0.5*((x - x0)**2 + (y - y0)**2)/sigma**2)

# Plot the z values:
ax.pcolormesh(x, y, z, cmap='reddish', vmin=0, vmax=1, shading='gouraud')
# some aesthetic things: plot things like ticks for the visual field:
for sp in ax.spines.values(): sp.set_visible(False)
tick_color = [0.75,0.75,0.75,1]
for etick in [1,2,4,8]:
    ax.add_patch(plt.Circle((0,0), etick, color=tick_color, lw=0.5, fill=False))
ax.plot([-8,8],[0,0], color=tick_color, lw=0.5)
ax.plot([0,0],[-8,8], color=tick_color, lw=0.5)
# add the text to the axis
ax.text(-8, 1, text, fontdict={'family':'monospace', 'horizontalalignment':'left'})
ax.axis('off');

In [ ]:
# Plot the predicted PRF response on top of the measured BOLD:
(fig,ax) = plt.subplots(1,1, figsize=(7,2.5), dpi=2*72)

# plot the prediction in red on top of the data in black:
ax.plot(np.arange(len(signal)), signal, 'k.-', lw=0.25, ms=0.5, marker='.')
pred = fit.prediction
ax.plot(np.arange(len(signal)), pred,   'r-', lw=1)

ax.set_xlabel('Time [s]')
ax.set_xlim([0,600])
ax.set_ylabel('BOLD Signal [AU]')
for k in ['top','right']: ax.spines[k].set_visible(False)
# Adjust the x-axis to be nicer for glancing at time:
ax.set_xticks(np.arange(0,601,300))
ax.xaxis.set_minor_locator(mpl.ticker.MultipleLocator(60))

## Retinotopic Maps

Here we look at the pRF solutions across the cortical surface: the retinotopic maps. The pRF data for all the subjects that participated in retinotopic mapping experiments in the HCP have already been solved and can be auto-downloaded by neuropythy. The OSF page for the HCP 7T retinotopy dataset project is [here](https://osf.io/bw9ec/), and the paper is [here](https://doi.org/10.1167/18.13.23).

The pRF parameters are saved as properties of the hemisphere objects of the subjects. There are 'lowres-prf_' and 'highres-prf_' properties as well as just 'prf_' properties (e.g., 'lowres-prf_polar_angle', 'prf_sigma'). The 'highres-prf_' properties are the results from the higher-resolution (59k) interpolation of the retinotopy data; the 'lowres-prf_' properties are the results solved on the 32k meshes. As of the time of this course, the high resolution results are not yet available and can be ignored (neuropythy will raise an exception if you try to use them). In the future, these will be added to the OSF and can be used then. The 'prf_' parameters are the 'highres-' parameters if they are available and otherwise the 'lowres-prf_' parameters.

In [ ]:
# Plot the pRF maps for the entire LH of the subject on the subject's native
# white surface; we'll also put a dot at the closest point to the vertex we
# previously solved

# In the following lines, we instruct neuropythy to plot the inflated surface
# (lh.white_surface is an alias for lh.surfaces['white']) and to color it
# according to the prf_polar_angle property. We also add a mask for the
# color overlay; a tuple (property, min, max) is one way to specify a mask;
# the mask could also be a binary mask or a list of vertex indices, or
# (property, (val1, val2, ...)).
# When neuropythy is given a color name that ends with polar_angle,
# eccentricity, or a variety of other recognized names, it will automatically
# pick a color schema; however, you can use the cmap and related matplotlib
# arguments here as well.
surf = 'inflated'
mesh = sub.lh.surfaces[surf]
fig = ny.cortex_plot(mesh,
                     color='prf_polar_angle',
                     mask=('prf_variance_explained', 0.04, np.inf))

ipv.show()

#### Compare to our solution

We can check how close our particular fit is to the fit found by analyzePRF (the HCP pRF results auto-loaded by neuropythy were solved using [analyzePRF](https://kendrickkay.net/analyzePRF/) by Kendrick Kay). To do this, we will pull the property off the nearest vertex we found above.

In [ ]:
# Get the parameters bundled in neuropythy for the nearest vertex:
fit = {k: sub.lh.prop('prf_'+v)[nearest]
       for (k,v) in zip(['x', 'y', 'sigma',  'beta', 'rsquared'],
                        ['x', 'y', 'radius', 'gain', 'variance_explained'])}
fit = ny.util.data_struct(fit)

In [ ]:
# We reproduce the pRF plot from above using these new parameters
text = r'''
PRF Parameters:
    (x0, y0): (%4.2f, %4.2f),
       sigma: %4.2f
        gain: %4.2f
   r-squared: %4.2f
''' % (fit.x, fit.y, fit.sigma, fit.beta, fit.rsquared)

# We'll now plot a visualization of the pRF itself...
(fig,ax) = plt.subplots(1,1, figsize=(7,7))

# Sample the visual field at points from -10°, to 10°:
max_eccen = 10
res = 250
samples = np.linspace(-max_eccen, max_eccen, res)
(x,y) = np.meshgrid(samples, samples)
# get the gaussian pRF sensitivity values:
(x0,y0,sigma) = (fit.x, fit.y, fit.sigma)
z = np.exp(-0.5*((x - x0)**2 + (y - y0)**2)/sigma**2)

# Plot the z values:
ax.pcolormesh(x, y, z, cmap='reddish', vmin=0, vmax=1, shading='gouraud')
# some aesthetic things: plot things like ticks for the visual field:
for sp in ax.spines.values(): sp.set_visible(False)
tick_color = [0.75,0.75,0.75,1]
for etick in [1,2,4,8]:
    ax.add_patch(plt.Circle((0,0), etick, color=tick_color, lw=0.5, fill=False))
ax.plot([-8,8],[0,0], color=tick_color, lw=0.5)
ax.plot([0,0],[-8,8], color=tick_color, lw=0.5)
# add the text to the axis
ax.text(-8, 1, text, fontdict={'family':'monospace', 'horizontalalignment':'left'})
ax.axis('off');

### Demo 7: Predict Retinotopy and/or Apply an Atlas

#### The Benson 2014 Atlas

In these cells, we apply the Benson 2014 atlas to a subject; this is a prediction, based entirely on the subject's anatomy, of the retinotopic maps in V1-V3 (as well as a number of other maps, but note that only the V1, V2, and V3 maps should be considered reliable--see [this paper](https://doi.org/10.7554/eLife.40224) for more information). The Benson 2014 maps include both V1, V2, and V3 ROI boundaries as well as polar angle and eccentricity predictions for every vertex.

In [ ]:
# make the prediction for both hemispheres:
benson14_rmaps = {h: ny.vision.predict_retinotopy(sub.hemis[h])
                  for h in ['lh','rh']}

In [ ]:
# Plot the polar angle prediction in V1-V3 for the LH:
ny.cortex_plot(sub.lh.surfaces['inflated'],
               color=benson14_rmaps['lh']['angle'],
               mask=(benson14_rmaps['lh']['varea'], (1,2,3)),
               # Since we didn't give a property name, we have to tell
               # neuropythy the colormap and min/max
               cmap='polar_angle', vmin=-180, vmax=180)
ipv.show()

#### The Wang 2015 Atlas

The Wang et al. (2015) probabilistic atlas of visual areas is a handy atlas for determining approximately where in the visual hierarchy a position on cortex is. It is based on the average boundary lines as drawn by hand across 53 subjects. See [this publication](https://doi.org/10.1093/cercor/bhu277) for additional details. Unlike the Benson 2014 maps, the Wang 2015 atlas provides information about visual area boundaries but does not include information about the organization of the retinotopic maps themselves.

Because the Wang atlas is defined on the fsaverage surface, we can interpolate from the fsaverage spherical surface onto a subject's fsaverage-registered spherical surface. Both FreeSurfer and HCP subjects' native hemispheres are usually aligned to the fsaverage hemispheres.

In [ ]:
# First, load the wang atlas; there is a copy bundled with neuropythy:
atlas_path = os.path.join(ny.util.library_path(), 'data', 'fsaverage', 'surf')
wang15_atlas = {h: ny.load(os.path.join(atlas_path, '%s.wang15_mplbl.v1_0.mgz' % h))
                for h in ['lh','rh']}
# The wang atlas is just a property (of area labels) for each vertex on the
# fsaverage hemisphere...

In [ ]:
# Now, interpolate from fsaverage over to our subject:
fsa = ny.freesurfer_subject('fsaverage')
# because the data are integers, this is automatically interpolated using nearest
# neighbor (though you can override with the method option); alignments to the
# fsaverage are detected automatically and used for surface interpolation
lh_wang15 = fsa.lh.interpolate(sub.lh, wang15_atlas['lh'])

# okay, now make a plot of the want atlas; we can make a custom label/annotation
# colormap using a utility function:
cm = ny.graphics.label_cmap(lh_wang15)
ny.cortex_plot(sub.lh.surfaces['inflated'], color=lh_wang15, cmap=cm)